In [ ]:
import asyncio
import json
import math
import sys
from datetime import date, datetime, timedelta

import alpaca_trade_api as tradeapi
import iso8601
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import pandas as pd
import pytz
import requests
from dateutil import parser
from IPython.display import HTML, display
from liualgotrader.analytics.analysis import (
    calc_batch_revenue,
    count_trades,
    load_trades_by_batch_id,
)
from liualgotrader.common import config
from liualgotrader.common.market_data import (
    get_historical_data_from_poylgon_for_symbols,
)
from pandas import DataFrame as df
from pytz import timezone
from talib import BBANDS, MACD, RSI

%matplotlib inline


nest_asyncio.apply()

In [ ]:
batch_id = "39f7770a-b2a6-455f-9e8d-7fdca3ab932f"

In [ ]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")

In [ ]:
trades = load_trades_by_batch_id(batch_id)

In [ ]:
day_to_analyze = min(trades["client_time"].tolist())
config.market_open = day_to_analyze.replace(hour=9, minute=30, second=0, microsecond=0)

In [ ]:
how_was_my_batch = pd.DataFrame()
trades["client_time"] = pd.to_datetime(trades["client_time"])
how_was_my_batch["symbol"] = trades.symbol.unique()
how_was_my_batch["revenues"] = how_was_my_batch["symbol"].apply(
    lambda x: calc_batch_revenue(x, trades, batch_id)
)
how_was_my_batch["count"] = how_was_my_batch["symbol"].apply(
    lambda x: count_trades(x, trades, batch_id)
)

In [ ]:
pd.set_option("display.max_rows", how_was_my_batch.shape[0] + 1)
how_was_my_batch

In [ ]:
print(f"Revenue: ${round(sum(how_was_my_batch['revenues'].tolist()), 2)}")

In [ ]:
minute_history = get_historical_data_from_poylgon_for_symbols(
    api,
    trades.symbol.unique().tolist(),
    day_to_analyze - timedelta(days=7),
    day_to_analyze + timedelta(days=1),
)

In [ ]:
for symbol in minute_history:
    symbol_df = trades.loc[trades["symbol"] == symbol]
    start_date = symbol_df["client_time"].min()
    start_date = start_date.replace(hour=9, minute=30)
    end_date = start_date.replace(hour=16, minute=0)
    cool_down_date = start_date + timedelta(minutes=5)
    minute_history_index = minute_history[symbol]["close"].index.get_loc(
        start_date, method="nearest"
    )
    end_index = minute_history[symbol]["close"].index.get_loc(
        end_date, method="nearest"
    )
    cool_minute_history_index = minute_history[symbol]["close"].index.get_loc(
        cool_down_date, method="nearest"
    )
    open_price = minute_history[symbol]["close"][cool_minute_history_index]
    plt.plot(
        minute_history[symbol]["close"][minute_history_index:end_index].between_time(
            "9:30", "16:00"
        ),
        label=symbol,
    )
    plt.xticks(rotation=45)

    delta = 0
    profit = 0

    operations = []
    deltas = []
    profits = []
    times = []
    prices = []
    qtys = []
    indicators = []
    target_price = []
    stop_price = []
    daily_change = []
    precent_vwap = []
    patterns = []
    algo_names = []
    for index, row in symbol_df.iterrows():
        if (
            row["operation"] == "buy"
            and row["qty"] > 0
            or row["operation"] == "sell"
            and row["qty"] < 0
        ):
            try:
                now = int(row["client_time"])
                continue
            except Exception:
                pass

        delta = (
            row["price"]
            * row["qty"]
            * (1 if row["operation"] == "sell" and row["qty"] > 0 else -1)
        )
        profit += delta
        plt.scatter(
            row["client_time"].to_pydatetime(),
            row["price"],
            c="g" if row["operation"] == "buy" else "r",
            s=100,
        )
        algo_names.append(row["algo_name"])
        deltas.append(round(delta, 2))
        profits.append(round(profit, 2))
        operations.append(row["operation"])
        times.append(pd.to_datetime(row["client_time"]))
        prices.append(row["price"])
        qtys.append(row["qty"])
        indicator = json.loads(row.indicators)
        indicators.append(indicator)
        target_price.append(row["target_price"])
        stop_price.append(row["stop_price"])
        daily_change.append(
            f"{round(100.0 * (float(row['price']) - open_price) / open_price, 2)}%"
        )
        precent_vwap.append(
            f"{round(100.0 * (indicator['buy']['avg'] - open_price) / open_price, 2)}%"
            if "buy" in indicator and indicator["buy"] and "avg" in indicator["buy"]
            else ""
        )

    d = {
        "profit": profits,
        "trade": deltas,
        "algo": algo_names,
        "operation": operations,
        "at": times,
        "price": prices,
        "qty": qtys,
        "daily change": daily_change,
        "vwap": precent_vwap,
        "indicators": indicators,
        "target price": target_price,
        "stop price": stop_price,
    }
    print(f"{symbol} analysis with profit {round(profit, 2)}")
    display(HTML(pd.DataFrame(data=d).to_html()))
    plt.legend()
    plt.show()